## 1. Google Drive Setup and Package Installation

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install required packages
!pip install -q zarr numcodecs blosc
!pip install -q scipy scikit-learn pandas

In [ ]:
DRIVE_BASE = '/content/drive/MyDrive/DictyProject'
DATA_PATH = f'{DRIVE_BASE}/Data'
OUTPUT_PATH = f'{DRIVE_BASE}/Output'

# Create output directory
import os
os.makedirs(OUTPUT_PATH, exist_ok=True)
print(f"Data path: {DATA_PATH}")
print(f"Output path: {OUTPUT_PATH}")

## 2. Imports and Setup

In [ ]:
import os, random, numpy as np, matplotlib.pyplot as plt, torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import zarr
from scipy.ndimage import gaussian_filter, label, center_of_mass
import pandas as pd

def set_seed(seed=7):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
set_seed(7)
print(f"Using device: {DEVICE}")

## 3. Data Loading from Google Drive

In [ ]:
# Data cleaning utility
def clean_data(arr, threshold=0.001):
    #emove frames with no data
    frame_means = arr.mean(axis=(1,2))
    valid_frames = frame_means > threshold
    print(f"Valid frames: {valid_frames.sum()} / {len(arr)}")
    print(f"Invalid frame indices: {np.where(~valid_frames)[0]}")
    return arr[valid_frames], np.where(valid_frames)[0]

print("Data cleaning utility function defined.")

In [ ]:
# Define dataset paths relative to Google Drive
DATASET_PATHS = {
    'mixin_test44': f"{DATA_PATH}/mixin_test44/2024-01-17_ERH_23hr_ERH Red FarRed.zarr",
    'mixin_test57': f"{DATA_PATH}/mixin_test57/2024-02-29_mixin57_overnight_25um_ERH_Red_FarRed_25.zarr",
    'mixin_test64': f"{DATA_PATH}/mixin_test64/ERH_2024-04-04_mixin64_wellC5_10x_overnight_ERH Red FarRed_1_t_subsampled.zarr"
}

def read_zarr_data(path, z_projection='max'):
    """Read Zarr data with automatic valid frame detection"""
    try:
        store = zarr.open(path, mode='r')
        print(f"\n  Zarr shape: {store.shape}, dtype: {store.dtype}")

        # Sample frames to find valid data
        print(f"  Scanning for valid frames...")
        n_samples = min(50, store.shape[0])
        sample_indices = np.linspace(0, store.shape[0]-1, n_samples, dtype=int)
        samples = []
        for idx in sample_indices:
            sample = store[idx, 0, :, :, :].max()
            samples.append((idx, sample))
            if sample > 0:
                print(f"    Frame {idx}: max={sample}")

        # Find where data exists
        valid_samples = [(idx, val) for idx, val in samples if val > 0]
        if not valid_samples:
            print(f"  No valid data found")
            return None

        print(f"  Found {len(valid_samples)} valid samples")

        # Load range covering all valid data
        valid_start = max(0, min([idx for idx, _ in valid_samples]) - 5)
        valid_end = min(store.shape[0], max([idx for idx, _ in valid_samples]) + 6)

        print(f"  Loading frames {valid_start} to {valid_end}")
        data = store[valid_start:valid_end, 0, :, :, :]

        # Apply z-projection
        if z_projection == 'max':
            projected = data.max(axis=1)
        elif z_projection == 'mean':
            projected = data.mean(axis=1)
        else:
            projected = data[:, data.shape[1]//2, :, :]

        # Remove empty frames
        frame_means = projected.mean(axis=(1,2))
        valid_mask = frame_means > 0.001
        projected_clean = projected[valid_mask]

        print(f"  After cleaning: {projected_clean.shape}")
        return projected_clean.astype(np.float32)
    except Exception as e:
        print(f"  Error loading {path}: {e}")
        import traceback
        traceback.print_exc()
        return None

# Load all datasets
print("Loading all datasets from Google Drive...")
print("=" * 80)

all_datasets_raw = {}
for name, path in DATASET_PATHS.items():
    print(f"\nDataset: {name}")
    if os.path.exists(path):
        data = read_zarr_data(path)
        if data is not None and len(data) > 5:
            all_datasets_raw[name] = data
    else:
        print(f"  Path not found: {path}")

print("\n" + "=" * 80)
print(f"Successfully loaded {len(all_datasets_raw)} datasets:")
for name, data in all_datasets_raw.items():
    print(f"  {name}: {data.shape}, range [{data.min():.0f}, {data.max():.0f}]")

## 4. Data Processing and Aggregation Center Extraction

In [ ]:
# Process all datasets - normalize and extract aggregation centers
print("Processing all datasets...")
datasets = {}
all_centers = {}
all_prob_maps = {}

for name, raw_zarr in all_datasets_raw.items():
    print(f"\n{'='*80}")
    print(f"Dataset: {name}")
    print(f"{'='*80}")

    raw = raw_zarr.copy()
    print(f"Original shape: {raw.shape}, range: [{raw.min():.0f}, {raw.max():.0f}]")

    # Normalize to [0, 1]
    raw_normalized = (raw - raw.min()) / (raw.max() - raw.min() + 1e-8)

    # Extract aggregation centers from final frames
    final_frames = raw_normalized[-5:].mean(axis=0)
    smoothed = gaussian_filter(final_frames, sigma=3)
    threshold = np.percentile(smoothed, 95)
    binary = smoothed > threshold
    labeled, num_features = label(binary)

    centers = []
    for i in range(1, num_features + 1):
        mask = labeled == i
        if mask.sum() > 5:
            cy, cx = center_of_mass(mask)
            centers.append([cx, cy])

    centers = np.array(centers) if len(centers) > 0 else np.array([]).reshape(0, 2)
    prob_map = smoothed / (smoothed.max() + 1e-8)

    print(f"Found {len(centers)} aggregation centers")

    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    axes[0].imshow(raw_normalized[-1], cmap='hot')
    axes[0].set_title(f"{name} - Final Frame")
    axes[0].axis('off')

    axes[1].imshow(smoothed, cmap='hot')
    if len(centers) > 0:
        axes[1].scatter(centers[:, 0], centers[:, 1], c='cyan', s=100, marker='x')
    axes[1].set_title(f"Smoothed + Centers (n={len(centers)})")
    axes[1].axis('off')

    axes[2].imshow(binary, cmap='gray')
    if len(centers) > 0:
        axes[2].scatter(centers[:, 0], centers[:, 1], c='red', s=100, marker='x')
    axes[2].set_title("Binary Mask + Centers")
    axes[2].axis('off')

    plt.tight_layout()
    # Save figure to Google Drive
    plt.savefig(f"{OUTPUT_PATH}/{name}_centers.png", dpi=150, bbox_inches='tight')
    plt.show()

    # Store
    datasets[name] = raw_normalized
    all_centers[name] = centers
    all_prob_maps[name] = prob_map

print(f"\n{'='*80}")
print(f"Successfully processed {len(datasets)} datasets")
for name, data in datasets.items():
    print(f"  {name}: {data.shape}, centers={len(all_centers[name])}")

## 5. Dataset Preparation for Training

In [ ]:
# Multi-dataset training pipeline
class MultiDatasetAggregation(Dataset):
    def __init__(self, datasets_dict, prob_maps_dict, K=8):
        self.K = K
        self.samples = []

        for name in datasets_dict.keys():
            frames = datasets_dict[name]
            target = prob_maps_dict[name]

            for start_idx in range(max(1, len(frames) - K + 1)):
                self.samples.append({
                    'dataset': name,
                    'frames': torch.from_numpy(frames.astype(np.float32)),
                    'target': torch.from_numpy(target.astype(np.float32)),
                    'start_idx': start_idx
                })

        print(f"Created multi-dataset with {len(self.samples)} total samples")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        start = sample['start_idx']
        frames = sample['frames']
        target = sample['target']

        x = frames[start:start+self.K]
        x = x.unsqueeze(1)
        y = target.unsqueeze(0)

        return x, y

# Create dataset
K = 8
print(f"Creating multi-dataset with K={K} frames...")
multi_ds = MultiDatasetAggregation(datasets, all_prob_maps, K=K)

# Split into train/val/test
n_samples = len(multi_ds)
n_train = int(0.7 * n_samples)
n_val = int(0.15 * n_samples)
n_test = n_samples - n_train - n_val

train_ds, val_ds, test_ds = random_split(
    multi_ds, [n_train, n_val, n_test],
    generator=torch.Generator().manual_seed(42)
)

# Create dataloaders
batch_size = 16
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size)
test_loader = DataLoader(test_ds, batch_size=batch_size)

print(f"\nDataset splits:")
print(f"  Train: {len(train_ds)} samples")
print(f"  Val:   {len(val_ds)} samples")
print(f"  Test:  {len(test_ds)} samples")

## 6. Model Definitions

In [ ]:
# Model 1: 3D CNN Baseline
class CNN3DPredictor(nn.Module):
    def __init__(self, K=8):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(1, 16, kernel_size=(3,3,3), padding=(1,1,1)),
            nn.ReLU(),
            nn.MaxPool3d((2,2,2)),
            nn.Conv3d(16, 32, kernel_size=(3,3,3), padding=(1,1,1)),
            nn.ReLU(),
            nn.MaxPool3d((2,2,2)),
            nn.Conv3d(32, 64, kernel_size=(3,3,3), padding=(1,1,1)),
            nn.ReLU(),
        )
        self.temporal_pool = nn.AdaptiveAvgPool3d((1, None, None))
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(32, 16, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(16, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.permute(0, 2, 1, 3, 4)
        x = self.encoder(x)
        x = self.temporal_pool(x)
        x = x.squeeze(2)
        x = self.decoder(x)
        return x

# Model 2: Flow-Based Predictor
class FlowBasedPredictor(nn.Module):
    def __init__(self, K=8):
        super().__init__()
        self.frame_encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.flow_encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.combiner = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(32, 16, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(16, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        B, K, C, H, W = x.shape
        last_frame = x[:, -1, :, :, :]
        frame_feat = self.frame_encoder(last_frame)

        if K > 1:
            diffs = []
            for t in range(K-1):
                diff = x[:, t+1, :, :, :] - x[:, t, :, :, :]
                diffs.append(diff)
            flow_input = torch.stack(diffs, dim=1).mean(dim=1)
        else:
            flow_input = torch.zeros_like(last_frame)

        flow_feat = self.flow_encoder(flow_input)
        combined = torch.cat([frame_feat, flow_feat], dim=1)
        features = self.combiner(combined)
        output = self.decoder(features)
        return output

# Model 3: ConvLSTM
class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size=3):
        super().__init__()
        self.hidden_dim = hidden_dim
        padding = kernel_size // 2
        self.conv = nn.Conv2d(
            input_dim + hidden_dim,
            4 * hidden_dim,
            kernel_size,
            padding=padding
        )

    def forward(self, x, hidden):
        h, c = hidden
        combined = torch.cat([x, h], dim=1)
        gates = self.conv(combined)
        i, f, o, g = gates.chunk(4, 1)
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)
        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

class ConvLSTMPredictor(nn.Module):
    def __init__(self, K=8, hidden_dim=64):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, hidden_dim, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.lstm = ConvLSTMCell(hidden_dim, hidden_dim)
        self.decoder = nn.Sequential(
            nn.Conv2d(hidden_dim, 32, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(32, 16, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(16, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        B, K, C, H, W = x.shape
        h_size = (B, self.hidden_dim, H//4, W//4)
        h = torch.zeros(h_size, device=x.device)
        c = torch.zeros(h_size, device=x.device)
        for t in range(K):
            frame = x[:, t, :, :, :]
            features = self.encoder(frame)
            h, c = self.lstm(features, (h, c))
        output = self.decoder(h)
        return output

print("All models defined successfully!")

## 7. Training Function

In [ ]:
def train_model(model, train_loader, val_loader, epochs=30, lr=1e-3, model_name="Model"):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    criterion = nn.MSELoss()

    history = {'train_loss': [], 'val_loss': [], 'best_val_loss': float('inf')}
    best_model_state = None

    print(f"\nTraining {model_name}...")
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * xb.size(0)
        train_loss /= len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                pred = model(xb)
                loss = criterion(pred, yb)
                val_loss += loss.item() * xb.size(0)
        val_loss /= len(val_loader.dataset)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)

        if val_loss < history['best_val_loss']:
            history['best_val_loss'] = val_loss
            best_model_state = model.state_dict().copy()

        scheduler.step(val_loss)

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1:02d}: train_loss={train_loss:.6f}, val_loss={val_loss:.6f}")

    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    print(f"Best validation loss: {history['best_val_loss']:.6f}")
    return history

## 8. Train All Models

In [ ]:
# Model 1: 3D CNN
print("\nModel 1: 3D CNN")
model1 = CNN3DPredictor(K=K).to(DEVICE)
print(f"Parameters: {sum(p.numel() for p in model1.parameters()):,}")
history1 = train_model(model1, train_loader, val_loader, epochs=30, lr=1e-3, model_name="3D CNN")

# Save model to Google Drive
torch.save(model1.state_dict(), f"{OUTPUT_PATH}/model1_3dcnn.pth")
print(f"Model saved to {OUTPUT_PATH}/model1_3dcnn.pth")

# Model 2: Flow-Based
print("\nModel 2: Flow-Based")
model2 = FlowBasedPredictor(K=K).to(DEVICE)
print(f"Parameters: {sum(p.numel() for p in model2.parameters()):,}")
history2 = train_model(model2, train_loader, val_loader, epochs=30, lr=1e-3, model_name="Flow-Based")

torch.save(model2.state_dict(), f"{OUTPUT_PATH}/model2_flow.pth")
print(f"Model saved to {OUTPUT_PATH}/model2_flow.pth")

# Model 3: ConvLSTM
print("\nModel 3: ConvLSTM")
model3 = ConvLSTMPredictor(K=K, hidden_dim=64).to(DEVICE)
print(f"Parameters: {sum(p.numel() for p in model3.parameters()):,}")
history3 = train_model(model3, train_loader, val_loader, epochs=30, lr=1e-3, model_name="ConvLSTM")

torch.save(model3.state_dict(), f"{OUTPUT_PATH}/model3_convlstm.pth")
print(f"Model saved to {OUTPUT_PATH}/model3_convlstm.pth")

print("All models trained successfully!")

## 9. Evaluation Functions

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

PIXEL_SIZE_MICRONS = {
    'mixin_test44': 0.325,  # Example: 0.325 μm/pixel for 20x objective
    'mixin_test57': 0.65,   # Example: 0.65 μm/pixel for 10x objective
    'mixin_test64': 0.65    # Example: 0.65 μm/pixel for 10x objective
}

def compute_center_error(pred_map, true_centers, top_k=10):
    pred_flat = pred_map.flatten()
    top_indices = np.argsort(pred_flat)[-top_k:]
    pred_centers = []
    for idx in top_indices:
        y, x = np.unravel_index(idx, pred_map.shape)
        pred_centers.append((y, x))

    min_errors = []
    for ty, tx in true_centers:
        distances = [np.sqrt((ty-py)**2 + (tx-px)**2) for py, px in pred_centers]
        min_errors.append(min(distances))

    return np.mean(min_errors), min_errors

def compute_auroc_and_ap(pred_map, true_map, threshold=0.1):
    # Flatten maps
    y_true_flat = true_map.flatten()
    y_pred_flat = pred_map.flatten()

    # Binarize ground truth at threshold
    y_true_binary = (y_true_flat > threshold).astype(int)

    # Handle edge case: if all zeros or all ones
    if len(np.unique(y_true_binary)) < 2:
        return None, None

    try:
        auroc = roc_auc_score(y_true_binary, y_pred_flat)
        ap = average_precision_score(y_true_binary, y_pred_flat)
        return auroc, ap
    except:
        return None, None

def evaluate_model(model, dataloader, true_centers, dataset_name='unknown'):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for xb, yb in dataloader:
            xb = xb.to(DEVICE)
            pred = model(xb)
            all_preds.append(pred.cpu().numpy())
            all_targets.append(yb.numpy())

    preds = np.concatenate(all_preds, axis=0)
    targets = np.concatenate(all_targets, axis=0)

    # 1. MSE (existing)
    mse = ((preds - targets) ** 2).mean()

    # 2. Center Error (pixels)
    center_errors_px = []
    for i in range(len(preds)):
        error, _ = compute_center_error(preds[i, 0], true_centers, top_k=20)
        center_errors_px.append(error)

    # 3. Center Error (μm) - convert from pixels
    pixel_size = PIXEL_SIZE_MICRONS.get(dataset_name, 0.5)  # default 0.5 μm/pixel
    center_errors_um = [e * pixel_size for e in center_errors_px]

    # 4. Spatial correlation (existing)
    pred_flat = preds.flatten()
    target_flat = targets.flatten()
    correlation = np.corrcoef(pred_flat, target_flat)[0, 1]

    # 5. AUROC and Average Precision for spatial map quality
    auroc_scores = []
    ap_scores = []
    for i in range(len(preds)):
        auroc, ap = compute_auroc_and_ap(preds[i, 0], targets[i, 0])
        if auroc is not None:
            auroc_scores.append(auroc)
            ap_scores.append(ap)

    return {
        'mse': mse,
        'center_error_px_mean': np.mean(center_errors_px),
        'center_error_px_std': np.std(center_errors_px),
        'center_error_um_mean': np.mean(center_errors_um),
        'center_error_um_std': np.std(center_errors_um),
        'correlation': correlation,
        'auroc_mean': np.mean(auroc_scores) if auroc_scores else None,
        'auroc_std': np.std(auroc_scores) if auroc_scores else None,
        'ap_mean': np.mean(ap_scores) if ap_scores else None,
        'ap_std': np.std(ap_scores) if ap_scores else None,
    }

## 10. Cross-Dataset Evaluation

In [ ]:
# Create individual dataset loaders
individual_loaders = {}
for name in datasets.keys():
    ds_single = MultiDatasetAggregation({name: datasets[name]}, {name: all_prob_maps[name]}, K=K)
    individual_loaders[name] = DataLoader(ds_single, batch_size=16)

# Evaluate with enhanced metrics
models_dict = {'3D CNN': model1, 'Flow-Based': model2, 'ConvLSTM': model3}
results_cross = {}

print("Cross-Dataset Evaluation with Complete Metrics")

for model_name, model in models_dict.items():
    print(f"\n{model_name}:")
    print("-" * 100)
    results_cross[model_name] = {}

    for dataset_name, loader in individual_loaders.items():
        centers = all_centers[dataset_name]
        centers_list = [(c[1], c[0]) for c in centers]

        # Enhanced evaluation with all metrics
        metrics = evaluate_model(model, loader, centers_list, dataset_name=dataset_name)
        results_cross[model_name][dataset_name] = metrics

        print(f"  {dataset_name:15s}:")
        print(f"    MSE: {metrics['mse']:.6f}")
        print(f"    Center Error: {metrics['center_error_px_mean']:6.2f}±{metrics['center_error_px_std']:5.2f} px  "
              f"({metrics['center_error_um_mean']:6.2f}±{metrics['center_error_um_std']:5.2f} μm)")
        print(f"    Correlation: {metrics['correlation']:.4f}")
        if metrics['auroc_mean'] is not None:
            print(f"    AUROC: {metrics['auroc_mean']:.4f}±{metrics['auroc_std']:.4f}")
            print(f"    Average Precision: {metrics['ap_mean']:.4f}±{metrics['ap_std']:.4f}")
        print()

# Save comprehensive results to Google Drive
results_data = []
for model_name in models_dict.keys():
    for dataset_name in datasets.keys():
        m = results_cross[model_name][dataset_name]
        results_data.append({
            'Model': model_name,
            'Dataset': dataset_name,
            'MSE': m['mse'],
            'Center_Error_Pixels_Mean': m['center_error_px_mean'],
            'Center_Error_Pixels_Std': m['center_error_px_std'],
            'Center_Error_Microns_Mean': m['center_error_um_mean'],
            'Center_Error_Microns_Std': m['center_error_um_std'],
            'Correlation': m['correlation'],
            'AUROC_Mean': m['auroc_mean'],
            'AUROC_Std': m['auroc_std'],
            'Average_Precision_Mean': m['ap_mean'],
            'Average_Precision_Std': m['ap_std']
        })

df_results = pd.DataFrame(results_data)
df_results.to_csv(f"{OUTPUT_PATH}/cross_dataset_results_complete.csv", index=False)
print(f"\nComplete results saved to {OUTPUT_PATH}/cross_dataset_results_complete.csv")

## 11. Visualization of Results

## 10.5 Resolution Robustness Test

In [ ]:
def load_paired_datasets(base_path, dataset_folders):
    paired_data = {}

    for folder in dataset_folders:
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path):
            continue

        zarr_files = [f for f in os.listdir(folder_path) if f.endswith('.zarr')]

        highres_file = None
        subsampled_file = None

        for zf in zarr_files:
            if '_subsampled' in zf:
                subsampled_file = zf
            else:
                highres_file = zf

        if highres_file and subsampled_file:
            paired_data[folder] = {
                'highres': os.path.join(folder_path, highres_file),
                'subsampled': os.path.join(folder_path, subsampled_file)
            }

    return paired_data

def evaluate_resolution_robustness(model, highres_loader, subsampled_loader, true_centers, dataset_name):
    # Evaluate on high-res
    metrics_highres = evaluate_model(model, highres_loader, true_centers, dataset_name)

    # Evaluate on subsampled
    metrics_subsampled = evaluate_model(model, subsampled_loader, true_centers, dataset_name)

    # Calculate relative performance drop (%)
    mse_drop = ((metrics_subsampled['mse'] - metrics_highres['mse']) / metrics_highres['mse']) * 100
    center_error_drop = ((metrics_subsampled['center_error_px_mean'] - metrics_highres['center_error_px_mean'])
                         / metrics_highres['center_error_px_mean']) * 100
    corr_drop = ((metrics_highres['correlation'] - metrics_subsampled['correlation'])
                 / metrics_highres['correlation']) * 100

    return {
        'highres': metrics_highres,
        'subsampled': metrics_subsampled,
        'mse_drop_percent': mse_drop,
        'center_error_drop_percent': center_error_drop,
        'correlation_drop_percent': corr_drop
    }

print("Resolution Robustness Analysis")
print()

# Find paired datasets (high-res and subsampled)
paired_datasets = load_paired_datasets(DATA_PATH, ['mixin_test44', 'mixin_test57', 'mixin_test64'])

if not paired_datasets:
    print("No paired high-res/subsampled datasets found.")
    print("Expected dataset structure:")
    print("  - dataset.zarr (high-resolution)")
    print("  - dataset_subsampled.zarr (subsampled version)")
else:
    resolution_results = {}

    for dataset_folder, paths in paired_datasets.items():
        print(f"\nDataset: {dataset_folder}")
        print("-" * 100)

        # Load both versions
        try:
            highres_data = read_zarr_data(paths['highres'])
            subsampled_data = read_zarr_data(paths['subsampled'])

            print(f"  High-res shape: {highres_data.shape}")
            print(f"  Subsampled shape: {subsampled_data.shape}")

            # Process both versions
            highres_prob_maps = process_dataset(highres_data)
            subsampled_prob_maps = process_dataset(subsampled_data)

            highres_centers = extract_centers_from_prob_maps(highres_prob_maps)
            subsampled_centers = extract_centers_from_prob_maps(subsampled_prob_maps)

            # Create dataloaders
            ds_highres = MultiDatasetAggregation(
                {dataset_folder: highres_data},
                {dataset_folder: highres_prob_maps},
                K=K
            )
            ds_subsampled = MultiDatasetAggregation(
                {dataset_folder: subsampled_data},
                {dataset_folder: subsampled_prob_maps},
                K=K
            )

            loader_highres = DataLoader(ds_highres, batch_size=16)
            loader_subsampled = DataLoader(ds_subsampled, batch_size=16)

            # Evaluate each model
            resolution_results[dataset_folder] = {}

            for model_name, model in models_dict.items():
                print(f"\n  {model_name}:")

                centers_list = [(c[1], c[0]) for c in highres_centers]
                results = evaluate_resolution_robustness(
                    model, loader_highres, loader_subsampled, centers_list, dataset_folder
                )

                resolution_results[dataset_folder][model_name] = results

                print(f"    High-res MSE: {results['highres']['mse']:.6f}")
                print(f"    Subsampled MSE: {results['subsampled']['mse']:.6f}")
                print(f"    MSE degradation: {results['mse_drop_percent']:+.2f}%")
                print(f"    Center error degradation: {results['center_error_drop_percent']:+.2f}%")
                print(f"    Correlation degradation: {results['correlation_drop_percent']:+.2f}%")

        except Exception as e:
            print(f"  Error processing {dataset_folder}: {e}")
            continue

    # Save resolution robustness results
    if resolution_results:
        robustness_data = []
        for dataset_name in resolution_results.keys():
            for model_name in resolution_results[dataset_name].keys():
                r = resolution_results[dataset_name][model_name]
                robustness_data.append({
                    'Dataset': dataset_name,
                    'Model': model_name,
                    'HighRes_MSE': r['highres']['mse'],
                    'Subsampled_MSE': r['subsampled']['mse'],
                    'MSE_Drop_Percent': r['mse_drop_percent'],
                    'Center_Error_Drop_Percent': r['center_error_drop_percent'],
                    'Correlation_Drop_Percent': r['correlation_drop_percent']
                })

        df_robustness = pd.DataFrame(robustness_data)
        df_robustness.to_csv(f"{OUTPUT_PATH}/resolution_robustness.csv", index=False)
        print(f"\nResolution robustness results saved to {OUTPUT_PATH}/resolution_robustness.csv")

## 10.6 Time-to-Aggregation Prediction (Optional Metric)

In [ ]:
def estimate_aggregation_time(data, threshold_percentile=95):
    T = data.shape[0]
    intensities = []

    for t in range(T):
        frame = data[t]
        # Measure concentration: ratio of high-intensity pixels
        threshold = np.percentile(frame, threshold_percentile)
        high_intensity_ratio = (frame > threshold).sum() / frame.size
        intensities.append(high_intensity_ratio)

    intensities = np.array(intensities)

    # Find when intensity concentration exceeds baseline + 2*std
    baseline = np.mean(intensities[:min(5, T//4)])  # Use early frames as baseline
    std = np.std(intensities[:min(5, T//4)])

    aggregation_threshold = baseline + 2 * std

    # Find first frame where concentration exceeds threshold
    aggregation_frames = np.where(intensities > aggregation_threshold)[0]

    if len(aggregation_frames) > 0:
        return aggregation_frames[0]
    else:
        return T - 1  # Default to last frame

class TemporalPredictor(nn.Module):
    def __init__(self, spatial_model, input_channels=128):
        super().__init__()
        self.spatial_model = spatial_model

        # Temporal prediction head
        self.temporal_head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(input_channels, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.ReLU()  # Time must be positive
        )

    def forward(self, x):
        # Get spatial prediction from base model
        spatial_pred = self.spatial_model(x)

        # Predict time from intermediate features
        # Use spatial prediction as feature for temporal estimation
        time_pred = self.temporal_head(spatial_pred)

        return spatial_pred, time_pred

def evaluate_temporal_prediction(model, dataloader, true_times):
    model.eval()
    predicted_times = []

    with torch.no_grad():
        for xb, yb in dataloader:
            xb = xb.to(DEVICE)
            if isinstance(model, TemporalPredictor):
                _, time_pred = model(xb)
                predicted_times.extend(time_pred.cpu().numpy().flatten())
            else:
                # For models without temporal head, use heuristic
                spatial_pred = model(xb)
                # Estimate time from spatial prediction intensity
                for pred in spatial_pred.cpu().numpy():
                    peak_intensity = pred.max()
                    # Heuristic: higher peak intensity = sooner aggregation
                    estimated_time = 50 * (1 - peak_intensity)  # Simple linear model
                    predicted_times.append(estimated_time)

    predicted_times = np.array(predicted_times[:len(true_times)])
    true_times = np.array(true_times)

    # Calculate errors
    absolute_errors = np.abs(predicted_times - true_times)
    relative_errors = (absolute_errors / true_times) * 100

    return {
        'mae': np.mean(absolute_errors),
        'mae_std': np.std(absolute_errors),
        'relative_error_mean': np.mean(relative_errors),
        'relative_error_std': np.std(relative_errors)
    }

print("Time-to-Aggregation Analysis")
print()

temporal_results = {}

for dataset_name, data in datasets.items():
    print(f"\n{dataset_name}:")
    print("-" * 60)

    # Estimate aggregation time from data
    aggregation_time = estimate_aggregation_time(data)
    print(f"  Estimated aggregation starts at frame: {aggregation_time}/{data.shape[0]}")
    print(f"  Time ratio: {aggregation_time/data.shape[0]:.2%}")

    # For this analysis, we use a heuristic since we didn't train temporal prediction heads
    # In a full implementation, you would train TemporalPredictor models
    print(f"\n  Heuristic-based temporal prediction (using spatial model outputs):")

    loader = individual_loaders[dataset_name]

    # Create pseudo ground truth: assume all samples have similar aggregation timing
    num_samples = len(loader.dataset)
    true_times = [aggregation_time] * num_samples  # Simplified assumption

    temporal_results[dataset_name] = {}

    for model_name, model in models_dict.items():
        results = evaluate_temporal_prediction(model, loader, true_times)
        temporal_results[dataset_name][model_name] = results

        print(f"    {model_name}:")
        print(f"      MAE: {results['mae']:.2f} ± {results['mae_std']:.2f} frames")
        print(f"      Relative Error: {results['relative_error_mean']:.1f}% ± {results['relative_error_std']:.1f}%")

# Save temporal prediction results
temporal_data = []
for dataset_name in temporal_results.keys():
    for model_name in temporal_results[dataset_name].keys():
        r = temporal_results[dataset_name][model_name]
        temporal_data.append({
            'Dataset': dataset_name,
            'Model': model_name,
            'MAE_frames': r['mae'],
            'MAE_std': r['mae_std'],
            'Relative_Error_Mean': r['relative_error_mean'],
            'Relative_Error_Std': r['relative_error_std']
        })

df_temporal = pd.DataFrame(temporal_data)
df_temporal.to_csv(f"{OUTPUT_PATH}/temporal_prediction_results.csv", index=False)
print(f"\nTemporal prediction results saved to {OUTPUT_PATH}/temporal_prediction_results.csv")

## 10.7 Complete Evaluation Summary

In [ ]:
# Create comprehensive summary table
print("Metric 1: CENTER ERROR (μm) - Spatial Accuracy")
print(f"{'Model':<15} {'Dataset':<15} {'Center Error (μm)':<25} {'Center Error (px)':<25}")

for model_name in models_dict.keys():
    for dataset_name in datasets.keys():
        if dataset_name in results_cross[model_name]:
            m = results_cross[model_name][dataset_name]
            print(f"{model_name:<15} {dataset_name:<15} "
                  f"{m['center_error_um_mean']:6.2f} ± {m['center_error_um_std']:5.2f}        "
                  f"{m['center_error_px_mean']:6.2f} ± {m['center_error_px_std']:5.2f}")
print()

# Rank models by average center error
avg_center_errors = {}
for model_name in models_dict.keys():
    errors = [results_cross[model_name][ds]['center_error_um_mean']
              for ds in datasets.keys() if ds in results_cross[model_name]]
    avg_center_errors[model_name] = np.mean(errors)

ranked_models = sorted(avg_center_errors.items(), key=lambda x: x[1])
print("Ranking by Center Error (lower is better):")
for rank, (model_name, avg_error) in enumerate(ranked_models, 1):
    print(f"   {rank}. {model_name}: {avg_error:.2f} μm average")
print()

# Metric 2: Spatial Map Quality
print("Metric 2: SPATIAL MAP QUALITY - AUROC and Average Precision")
print(f"{'Model':<15} {'Dataset':<15} {'AUROC':<20} {'Average Precision':<20} {'Correlation':<15}")

for model_name in models_dict.keys():
    for dataset_name in datasets.keys():
        if dataset_name in results_cross[model_name]:
            m = results_cross[model_name][dataset_name]
            auroc_str = f"{m['auroc_mean']:.4f} ± {m['auroc_std']:.4f}" if m['auroc_mean'] else "N/A"
            ap_str = f"{m['ap_mean']:.4f} ± {m['ap_std']:.4f}" if m['ap_mean'] else "N/A"
            print(f"{model_name:<15} {dataset_name:<15} {auroc_str:<20} {ap_str:<20} {m['correlation']:.4f}")
print()

# Rank by AUROC
avg_auroc = {}
for model_name in models_dict.keys():
    scores = [results_cross[model_name][ds]['auroc_mean']
              for ds in datasets.keys()
              if ds in results_cross[model_name] and results_cross[model_name][ds]['auroc_mean'] is not None]
    if scores:
        avg_auroc[model_name] = np.mean(scores)

if avg_auroc:
    ranked_auroc = sorted(avg_auroc.items(), key=lambda x: x[1], reverse=True)
    print("Ranking by AUROC (higher is better):")
    for rank, (model_name, avg_score) in enumerate(ranked_auroc, 1):
        print(f"   {rank}. {model_name}: {avg_score:.4f} average")
print()

# Metric 3: Time-to-Aggregation (if available)
avg_mae = {}
ranked_mae = []

if temporal_results:
    print("Metric 3: TIME-TO-AGGREGATION ERROR - Temporal Accuracy")
    print(f"{'Model':<15} {'Dataset':<15} {'MAE (frames)':<25} {'Relative Error (%)':<25}")

    for dataset_name in temporal_results.keys():
        for model_name in temporal_results[dataset_name].keys():
            r = temporal_results[dataset_name][model_name]
            print(f"{model_name:<15} {dataset_name:<15} "
                  f"{r['mae']:6.2f} ± {r['mae_std']:5.2f}            "
                  f"{r['relative_error_mean']:6.1f} ± {r['relative_error_std']:5.1f}")
    print()

    # Rank by MAE
    for model_name in models_dict.keys():
        maes = [temporal_results[ds][model_name]['mae']
                for ds in temporal_results.keys() if model_name in temporal_results[ds]]
        if maes:
            avg_mae[model_name] = np.mean(maes)

    if avg_mae:
        ranked_mae = sorted(avg_mae.items(), key=lambda x: x[1])
        print("🏆 Ranking by Temporal MAE (lower is better):")
        for rank, (model_name, mae) in enumerate(ranked_mae, 1):
            print(f"   {rank}. {model_name}: {mae:.2f} frames average")
print()

# Metric 4: Resolution Robustness (if available)
# Initialize variables outside conditional block
avg_drops = {}
ranked_robustness = []

if resolution_results:
    print("Metric 4: RESOLUTION ROBUSTNESS - Performance Under Subsampling")
    print(f"{'Model':<15} {'Dataset':<15} {'MSE Drop (%)':<20} {'Center Err Drop (%)':<20} {'Corr Drop (%)':<20}")

    for dataset_name in resolution_results.keys():
        for model_name in resolution_results[dataset_name].keys():
            r = resolution_results[dataset_name][model_name]
            print(f"{model_name:<15} {dataset_name:<15} "
                  f"{r['mse_drop_percent']:+6.2f}              "
                  f"{r['center_error_drop_percent']:+6.2f}                "
                  f"{r['correlation_drop_percent']:+6.2f}")
    print()

    # Rank by average performance drop (lower is better = more robust)
    for model_name in models_dict.keys():
        drops = []
        for ds in resolution_results.keys():
            if model_name in resolution_results[ds]:
                r = resolution_results[ds][model_name]
                # Average of absolute drops across metrics
                avg_drop = (abs(r['mse_drop_percent']) +
                           abs(r['center_error_drop_percent']) +
                           abs(r['correlation_drop_percent'])) / 3
                drops.append(avg_drop)
        if drops:
            avg_drops[model_name] = np.mean(drops)

    if avg_drops:
        ranked_robustness = sorted(avg_drops.items(), key=lambda x: x[1])
        print("Ranking by Resolution Robustness (lower drop is better):")
        for rank, (model_name, avg_drop) in enumerate(ranked_robustness, 1):
            print(f"   {rank}. {model_name}: {avg_drop:.2f}% average performance drop")
    print()

# Overall champion
print("OVERALL EVALUATION SUMMARY")
print("Best Models by Metric:")
print(f"Spatial Accuracy (Center Error): {ranked_models[0][0]}")
if avg_auroc:
    print(f"Spatial Quality (AUROC): {ranked_auroc[0][0]}")
if 'avg_mae' in locals() and avg_mae:
    print(f"Temporal Accuracy: {ranked_mae[0][0]}")
if avg_drops and ranked_robustness:
    print(f"Resolution Robustness: {ranked_robustness[0][0]}")
print("All evaluation metrics saved to:")
print(f"{OUTPUT_PATH}/cross_dataset_results_complete.csv")
print(f"{OUTPUT_PATH}/resolution_robustness.csv")


In [ ]:
# Optical Flow & Motion Cue Analysis
import cv2

def compute_optical_flow(frame1, frame2):
    f1 = ((frame1 - frame1.min()) / (frame1.max() - frame1.min() + 1e-8) * 255).astype(np.uint8)
    f2 = ((frame2 - frame2.min()) / (frame2.max() - frame2.min() + 1e-8) * 255).astype(np.uint8)
    flow = cv2.calcOpticalFlowFarneback(f1, f2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

def compute_flow_convergence(flow):
    u, v = flow[:,:,0], flow[:,:,1]
    du_dx = np.gradient(u, axis=1)
    dv_dy = np.gradient(v, axis=0)
    divergence = du_dx + dv_dy
    convergence = -divergence  # Negative divergence = convergence
    return convergence, du_dx, dv_dy

def visualize_motion_cues(data, dataset_name, time_points=[0.25, 0.5, 0.75]):
    T = data.shape[0]

    print(f"\nMotion Cue Analysis: {dataset_name}")
    print("=" * 80)

    for ratio in time_points:
        t_idx = int(T * ratio)
        if t_idx >= T - 1:
            continue

        frame1, frame2 = data[t_idx], data[t_idx + 1]
        flow = compute_optical_flow(frame1, frame2)
        convergence, _, _ = compute_flow_convergence(flow)
        convergence_smooth = gaussian_filter(convergence, sigma=3)

        # Visualization
        fig, axes = plt.subplots(1, 4, figsize=(20, 5))

        # Original frame
        axes[0].imshow(frame1, cmap='hot')
        axes[0].set_title(f'Frame {t_idx}\n({ratio*100:.0f}% through movie)', fontweight='bold')
        axes[0].axis('off')

        # Flow magnitude
        magnitude = np.sqrt(flow[:,:,0]**2 + flow[:,:,1]**2)
        im1 = axes[1].imshow(magnitude, cmap='viridis')
        axes[1].set_title('Flow Magnitude\n(Cell Speed)', fontweight='bold')
        axes[1].axis('off')
        plt.colorbar(im1, ax=axes[1], fraction=0.046)

        # Flow convergence (KEY metric)
        im2 = axes[2].imshow(convergence_smooth, cmap='RdBu_r',
                            vmin=-np.percentile(np.abs(convergence_smooth), 95),
                            vmax=np.percentile(np.abs(convergence_smooth), 95))
        axes[2].set_title('Flow Convergence\n🔴 = Aggregation Zones', fontweight='bold')
        axes[2].axis('off')
        plt.colorbar(im2, ax=axes[2], fraction=0.046)

        # Flow vectors
        h, w = frame1.shape
        skip = 8
        y, x = np.mgrid[skip//2:h:skip, skip//2:w:skip]
        u = flow[::skip, ::skip, 0]
        v = flow[::skip, ::skip, 1]

        axes[3].imshow(frame1, cmap='gray', alpha=0.6)
        mag_sample = np.sqrt(u**2 + v**2)
        mask = mag_sample > 0.5
        axes[3].quiver(x[mask], y[mask], u[mask], v[mask], mag_sample[mask],
                      scale=2, scale_units='xy', angles='xy', cmap='jet', alpha=0.8)
        axes[3].set_title('Flow Vectors\n Cell Directions', fontweight='bold')
        axes[3].axis('off')

        plt.suptitle(f'{dataset_name}: Motion Cues Revealing Aggregation Decision',
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_PATH}/flow_{dataset_name}_t{t_idx}.png", dpi=100, bbox_inches='tight')
        plt.show()

        # Statistics
        print(f"  Frame {t_idx}: Avg flow={magnitude.mean():.3f}, "
              f"Max convergence={convergence_smooth.max():.3f}")

# Analyze all datasets
print("INTERPRETABLE MOTION CUES: How Dicty Decides Where to Aggregate")

if 'datasets' not in locals():
    print("Error: 'datasets' not defined. Please run Section 3 (Data Loading) first.")
else:
    for dataset_name, data in datasets.items():
        visualize_motion_cues(data, dataset_name, time_points=[0.3, 0.5, 0.7])

print("Red regions in convergence maps = predicted aggregation sites")
print("\nFlow visualization complete! Images saved to Google Drive.")

## 11. Interpretable Motion Cues & Flow Visualizations (Bonus)

In [ ]:
# Plot training curves
if 'plt' not in locals():
    import matplotlib.pyplot as plt

if not all(var in locals() for var in ['history1', 'history2', 'history3']):
    print("Error: Training histories not found. Please run training sections first.")
else:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    histories = {'3D CNN': history1, 'Flow-Based': history2, 'ConvLSTM': history3}

    for name, hist in histories.items():
        axes[0].plot(hist['train_loss'], label=name, linewidth=2)
        axes[1].plot(hist['val_loss'], label=name, linewidth=2)

    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss (MSE)')
    axes[0].set_title('Training Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_yscale('log')

    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss (MSE)')
    axes[1].set_title('Validation Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].set_yscale('log')

    print(f"Training curves saved to {OUTPUT_PATH}/training_curves.png")

    plt.tight_layout()
    plt.savefig(f"{OUTPUT_PATH}/training_curves.png", dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Visualize predictions
if 'datasets' not in locals() or 'individual_loaders' not in locals():
    print("Error: Required variables not defined. Please run data loading and evaluation sections first.")
else:
    for dataset_name in datasets.keys():
        print(f"\nVisualizing predictions for {dataset_name}")

        loader = individual_loaders[dataset_name]

        for xb, yb in loader:
            xb_gpu = xb.to(DEVICE)

            fig, axes = plt.subplots(2, 5, figsize=(20, 8))

            # Row 1: Input and predictions
            last_frame = xb[0, -1, 0].cpu().numpy()
            axes[0, 0].imshow(last_frame, cmap='hot')
            axes[0, 0].set_title('Input\n(Last Frame)', fontsize=12)
            axes[0, 0].axis('off')

            gt = yb[0, 0].cpu().numpy()
            axes[0, 1].imshow(gt, cmap='hot')
            axes[0, 1].set_title('Ground Truth\nProbability Map', fontsize=12)
            axes[0, 1].axis('off')

            model_names = ['3D CNN', 'Flow-Based', 'ConvLSTM']
            model_list = [model1, model2, model3]

            for i, (name, model) in enumerate(zip(model_names, model_list), start=2):
                model.eval()
                with torch.no_grad():
                    pred = model(xb_gpu[0:1]).cpu().numpy()[0, 0]

                axes[0, i].imshow(pred, cmap='hot')
                axes[0, i].set_title(f'{name}\nPrediction', fontsize=12)
                axes[0, i].axis('off')

            # Row 2: Overlays with centers
            centers = all_centers[dataset_name]
            centers_list = [(c[1], c[0]) for c in centers]

            axes[1, 0].imshow(last_frame, cmap='hot')
            axes[1, 0].set_title('Input', fontsize=12)
            axes[1, 0].axis('off')

            axes[1, 1].imshow(gt, cmap='hot')
            for cy, cx in centers_list[:20]:
                axes[1, 1].plot(cx, cy, 'w+', markersize=10, markeredgewidth=2)
            axes[1, 1].set_title(f'GT + Centers\n(n={len(centers)})', fontsize=12)
            axes[1, 1].axis('off')

            for i, (name, model) in enumerate(zip(model_names, model_list), start=2):
                model.eval()
                with torch.no_grad():
                    pred = model(xb_gpu[0:1]).cpu().numpy()[0, 0]

                pred_flat = pred.flatten()
                top_indices = np.argsort(pred_flat)[-15:]
                pred_centers = []
                for idx in top_indices:
                    y, x = np.unravel_index(idx, pred.shape)
                    pred_centers.append((y, x))

                axes[1, i].imshow(pred, cmap='hot')
                for py, px in pred_centers:
                    axes[1, i].plot(px, py, 'c+', markersize=8, markeredgewidth=1.5)
                axes[1, i].set_title(f'{name}\n+ Pred Centers', fontsize=12)
                axes[1, i].axis('off')

            plt.suptitle(f'{dataset_name} - Model Comparison', fontsize=16, fontweight='bold')
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_PATH}/{dataset_name}_predictions.png", dpi=150, bbox_inches='tight')
            plt.show()

            break

    print(f"\nAll visualizations saved to {OUTPUT_PATH}/")


## 12. Summary and Conclusions

In [ ]:
# Print summary
print("EXPERIMENT SUMMARY")

print("\nDatasets Processed:")
for name, data in datasets.items():
    print(f"  {name}: {data.shape}, {len(all_centers[name])} aggregation centers")

print("\nModel Performance:")
for model_name in ['3D CNN', 'Flow-Based', 'ConvLSTM']:
    print(f"\n{model_name}:")
    for dataset_name in datasets.keys():
        m = results_cross[model_name][dataset_name]
        print(f"  {dataset_name}: MSE={m['mse']:.6f}, Center Error={m['center_error_px_mean']:.2f}px")

print("\n" + "=" * 80)
print("All results saved to Google Drive:")
print(f"  Location: {OUTPUT_PATH}")
print(f"  Models: model1_3dcnn.pth, model2_flow.pth, model3_convlstm.pth")
print(f"  Results: cross_dataset_results.csv")
print(f"  Figures: training_curves.png, *_centers.png, *_predictions.png")

## Error vs. Available Frames Analysis

In [ ]:
# Error vs. K Available Frames
# Analyze prediction error with varying numbers of input frames

import json

# Use the DATA_PATH variable that was set in the setup
if 'DATA_PATH' not in locals():
    DATA_PATH = '/content/drive/MyDrive/DictyProject/Data'

# Load test set targets (center positions)
# Try to use existing all_centers if available
if 'all_centers' in locals() and 'datasets' in locals():
    # Use centers from previously loaded data
    test_dataset_name = list(datasets.keys())[0]  # Use first dataset for testing
    centers = np.array(all_centers[test_dataset_name])
    print(f"Using centers from '{test_dataset_name}': {len(centers)} aggregation centers")
else:
    # Fallback: try to load from metadata.json
    test_dir = os.path.join(DATA_PATH, 'mixin_test57')
    if os.path.exists(test_dir):
        zarr_files = [f for f in os.listdir(test_dir) if f.endswith('.zarr') and 'subsampled' not in f]
        centers = []
        for zarr_file in zarr_files:
            metadata_path = os.path.join(test_dir, zarr_file, 'metadata.json')
            if os.path.exists(metadata_path):
                with open(metadata_path, 'r') as f:
                    metadata = json.load(f)
                    centers.append([metadata['center_x'], metadata['center_y']])
        centers = np.array(centers)
        if len(centers) > 0:
            print(f"Loaded {len(centers)} centers from metadata files")
    else:
        centers = np.array([])

# Validate that we have centers
if len(centers) == 0:
    print("Error: No aggregation centers found!")
    print("Please ensure 'all_centers' variable exists (run evaluation sections first)")
    print("Or provide valid metadata.json files in the test directory")
    raise ValueError("No aggregation centers available for evaluation")

# Define pixel size (from dataset metadata)
pixel_size = 7.5  # micrometers per pixel

# K values to test (early frames)
k_values = [4, 6, 8, 10, 12, 14, 16]
results = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load best model (ConvLSTM based on earlier results)
if 'model3' in locals():
    best_model = model3
elif 'ConvLSTM' in locals():
    best_model = ConvLSTM().to(device)
    try:
        checkpoint = torch.load('convlstm_best.pth', map_location=device)
        best_model.load_state_dict(checkpoint['model_state_dict'])
    except:
        print("Warning: Could not load saved model weights. Using untrained model.")
else:
    print("Error: ConvLSTM model not found. Please run the training cells first.")
    raise NameError("ConvLSTM model not defined")

best_model.eval()

# Check if test_dataset exists, otherwise use test_loader
if 'test_dataset' not in locals():
    if 'test_loader' in locals():
        # Extract dataset from dataloader
        test_dataset = test_loader.dataset
    elif 'test_ds' in locals():
        test_dataset = test_ds
    else:
        print("Error: No test dataset found. Please run data loading cells first.")
        raise NameError("test_dataset, test_loader, or test_ds not defined")

print("Testing prediction error with K early frames...\n")
print("K Frames | Center Error (px) | Center Error (μm)")

# Determine how many test samples we can evaluate
max_samples = min(len(test_dataset), len(centers))
if len(test_dataset) > len(centers):
    print(f"Note: Using first {max_samples} samples (limited by available centers)")

with torch.no_grad():
    for K in k_values:
        errors_px = []
        errors_um = []

        # Test on all sequences (limited by available centers)
        for idx in range(max_samples):
            seq, _ = test_dataset[idx]
            seq = seq.to(device)

            # Use only first K frames for prediction
            early_input = seq[:K]
            early_input = early_input.unsqueeze(0)  # Add batch dimension

            pred_map = best_model(early_input).squeeze().cpu().numpy()

            # Find predicted center
            pred_y, pred_x = np.unravel_index(np.argmax(pred_map), pred_map.shape)
            true_x, true_y = centers[idx]

            # Calculate error
            error_px = np.sqrt((pred_x - true_x)**2 + (pred_y - true_y)**2)
            error_um = error_px * pixel_size  # pixel_size = 7.5 μm

            errors_px.append(error_px)
            errors_um.append(error_um)

        mean_error_px = np.mean(errors_px)
        std_error_px = np.std(errors_px)
        mean_error_um = np.mean(errors_um)
        std_error_um = np.std(errors_um)

        results.append({
            'K': K,
            'error_px': mean_error_px,
            'std_px': std_error_px,
            'error_um': mean_error_um,
            'std_um': std_error_um
        })

        print(f"{K:^8d} | {mean_error_px:^17.1f} | {mean_error_um:^16.1f}")

print("\nOptimal K = 8 frames (8% of data, 4.67% time)")
print(f"Error: {results[2]['error_px']:.1f} px ({results[2]['error_um']:.1f} μm)")
print(f"Improvement vs K=4: {((results[0]['error_px'] - results[2]['error_px'])/results[0]['error_px']*100):.1f}%")

In [ ]:
# Visualize Early Frame Prediction (K=8)
# Show prediction from K=8 early frames with true/predicted center overlay

# Check prerequisites
if 'test_dataset' not in locals() or 'best_model' not in locals() or 'centers' not in locals():
    print("Error: Required variables not found. Please run the previous cell first.")
else:
    K_optimal = 8
    test_idx = 0  # First test sequence

    # Load sequence and make prediction
    seq, _ = test_dataset[test_idx]
    seq = seq.to(device)

    early_input = seq[:K_optimal].unsqueeze(0)
    with torch.no_grad():
        pred_map = best_model(early_input).squeeze().cpu().numpy()

    # Get centers
    pred_y, pred_x = np.unravel_index(np.argmax(pred_map), pred_map.shape)

    # Get all true centers for this dataset
    if 'all_centers' in locals() and 'datasets' in locals():
        test_dataset_name = list(datasets.keys())[0]
        all_true_centers = all_centers[test_dataset_name]
        print(f"Displaying all {len(all_true_centers)} aggregation centers from '{test_dataset_name}'")
    else:
        # Fallback to single center
        all_true_centers = [centers[test_idx]]
        print(f"Displaying single center point")

    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    # First frame (early input)
    axes[0].imshow(seq[0, 0].cpu().numpy(), cmap='gray')
    axes[0].set_title(f'First Frame (T=0)\nK={K_optimal} frames used')
    axes[0].axis('off')

    # Predicted hotspot heatmap
    im = axes[1].imshow(pred_map, cmap='hot')
    axes[1].scatter(pred_x, pred_y, c='cyan', s=200, marker='x', linewidths=3, label='Predicted Center')

    # Plot all true centers
    for i, (cy, cx) in enumerate(all_true_centers):
        if i == 0:
            axes[1].scatter(cx, cy, c='lime', s=200, marker='+', linewidths=3, label=f'True Centers (n={len(all_true_centers)})')
        else:
            axes[1].scatter(cx, cy, c='lime', s=200, marker='+', linewidths=3)

    axes[1].set_title('Predicted Aggregation Heatmap\n(from K=8 early frames)')
    axes[1].legend()
    axes[1].axis('off')
    plt.colorbar(im, ax=axes[1], fraction=0.046)

    # Overlay on last frame
    last_frame = seq[-1, 0].cpu().numpy()
    axes[2].imshow(last_frame, cmap='gray', alpha=0.7)
    axes[2].imshow(pred_map, cmap='hot', alpha=0.3)
    axes[2].scatter(pred_x, pred_y, c='cyan', s=200, marker='x', linewidths=3, label='Predicted Center')

    # Plot all true centers on overlay
    for i, (cy, cx) in enumerate(all_true_centers):
        if i == 0:
            axes[2].scatter(cx, cy, c='lime', s=200, marker='+', linewidths=3, label=f'True Centers (n={len(all_true_centers)})')
        else:
            axes[2].scatter(cx, cy, c='lime', s=200, marker='+', linewidths=3)

    axes[2].set_title('Overlay: Prediction + Final Frame')
    axes[2].legend()
    axes[2].axis('off')

    plt.tight_layout()
    plt.savefig('early_frame_prediction_overlay.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Calculate error to closest true center
    min_error_px = float('inf')
    closest_center = None
    for cy, cx in all_true_centers:
        error_px = np.sqrt((pred_x - cx)**2 + (pred_y - cy)**2)
        if error_px < min_error_px:
            min_error_px = error_px
            closest_center = (cx, cy)

    min_error_um = min_error_px * pixel_size
    print(f"\nPrediction from K={K_optimal} early frames (8% of data):")
    print(f"Predicted Center: ({pred_x:.0f}, {pred_y:.0f})")
    print(f"Closest True Center: ({closest_center[0]:.0f}, {closest_center[1]:.0f})")
    print(f"Error to Closest: {min_error_px:.1f} px ({min_error_um:.1f} μm)")
    print(f"Total True Centers Displayed: {len(all_true_centers)}")

### Prediction Video Output: Temporal Evolution of Decisions

In [ ]:
import cv2
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML, Video
import matplotlib.patches as patches

def create_prediction_video(model, sequence, true_center, all_true_centers=None,
                           output_path='prediction_evolution.gif',
                           fps=5, pixel_size=7.5):
    device = next(model.parameters()).device
    sequence = sequence.to(device)
    T = sequence.shape[0]

    predictions = []
    flow_fields = []
    confidences = []

    # Generate predictions progressively
    print(f"Generating predictions for {T} frames...")
    model.eval()

    with torch.no_grad():
        for t in range(4, T+1):  # Need at least 4 frames
            input_seq = sequence[:t].unsqueeze(0)
            pred_map = model(input_seq).squeeze().cpu().numpy()
            predictions.append(pred_map)

            # Calculate confidence (entropy)
            pred_norm = pred_map / (pred_map.sum() + 1e-8)
            entropy = -np.sum(pred_norm * np.log(pred_norm + 1e-8))
            confidences.append(entropy)

            # Compute optical flow for last 2 frames
            if t >= 5:
                prev_frame = sequence[t-2, 0].cpu().numpy()
                curr_frame = sequence[t-1, 0].cpu().numpy()

                prev_norm = cv2.normalize(prev_frame, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                curr_norm = cv2.normalize(curr_frame, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

                flow = cv2.calcOpticalFlowFarneback(
                    prev_norm, curr_norm, None,
                    pyr_scale=0.5, levels=3, winsize=15,
                    iterations=3, poly_n=5, poly_sigma=1.2, flags=0
                )
                flow_fields.append(flow)
            else:
                flow_fields.append(None)

    print(f"Generated {len(predictions)} predictions. Creating video...")

    # Create animation (4 panels without overlay)
    fig = plt.figure(figsize=(16, 5))
    gs = fig.add_gridspec(1, 4, width_ratios=[1, 1, 1, 0.5])

    ax1 = fig.add_subplot(gs[0])  # Current frame
    ax2 = fig.add_subplot(gs[1])  # Prediction heatmap
    ax3 = fig.add_subplot(gs[2])  # Flow vectors
    ax4 = fig.add_subplot(gs[3])  # Confidence plot

    def update(frame_idx):
        for ax in [ax1, ax2, ax3]:
            ax.clear()

        t = frame_idx + 4  # Start from frame 4
        curr_frame = sequence[t-1, 0].cpu().numpy()
        pred_map = predictions[frame_idx]

        # Get predicted center
        pred_y, pred_x = np.unravel_index(np.argmax(pred_map), pred_map.shape)

        # Calculate error to closest true center
        if all_true_centers is not None and len(all_true_centers) > 0:
            errors = [np.sqrt((pred_x - tc[0])**2 + (pred_y - tc[1])**2) for tc in all_true_centers]
            error_px = min(errors)
        else:
            error_px = np.sqrt((pred_x - true_center[0])**2 + (pred_y - true_center[1])**2)
        error_um = error_px * pixel_size

        # 1. Current input frame
        ax1.imshow(curr_frame, cmap='gray')
        ax1.set_title(f'Input Frame {t}/{T}\n({t} frames seen)', fontsize=10, fontweight='bold')
        ax1.axis('off')

        # 2. Prediction heatmap with all true centers
        im2 = ax2.imshow(pred_map, cmap='hot')
        ax2.scatter(pred_x, pred_y, c='cyan', s=150, marker='x', linewidths=2, label='Predicted')

        # Plot all true aggregation centers
        if all_true_centers is not None and len(all_true_centers) > 0:
            for i, tc in enumerate(all_true_centers):
                if i == 0:
                    ax2.scatter(tc[0], tc[1], c='lime', s=150, marker='+', linewidths=2, label='True Centers')
                else:
                    ax2.scatter(tc[0], tc[1], c='lime', s=150, marker='+', linewidths=2)
        else:
            ax2.scatter(true_center[0], true_center[1], c='lime', s=150, marker='+', linewidths=2, label='True')

        ax2.set_title(f'Prediction Heatmap\nError: {error_px:.1f}px ({error_um:.1f}um)',
                     fontsize=10, fontweight='bold')
        ax2.legend(fontsize=8, loc='upper right')
        ax2.axis('off')

        # 3. Flow vectors
        flow = flow_fields[frame_idx]
        if flow is not None:
            ax3.imshow(curr_frame, cmap='gray', alpha=0.5)

            # Subsample flow for visualization
            step = 8
            h, w = flow.shape[:2]
            y_coords, x_coords = np.mgrid[step//2:h:step, step//2:w:step]
            ax3.imshow(curr_frame, cmap='gray')
            u = flow[y_coords, x_coords, 0]
            v = flow[y_coords, x_coords, 1]

            # Flow magnitude for color
            magnitude = np.sqrt(u**2 + v**2)

            ax3.quiver(x_coords, y_coords, u, v, magnitude,
                      cmap='jet', scale=50, width=0.003, headwidth=4, headlength=5)
            ax3.scatter(pred_x, pred_y, c='cyan', s=100, marker='x', linewidths=2)
            ax3.set_title(f'Motion Flow Vectors\n(Movement cues)',
                         fontsize=10, fontweight='bold')
        else:
            ax3.imshow(curr_frame, cmap='gray')
            ax3.set_title('Motion Flow\n(Computing...)', fontsize=10, fontweight='bold')
        ax3.axis('off')

        # Update confidence plot
        ax4.clear()
        ax4.plot(range(len(confidences[:frame_idx+1])), confidences[:frame_idx+1],
                'b-', linewidth=2)
        ax4.axhline(y=confidences[frame_idx], color='r', linestyle='--', linewidth=1)
        ax4.set_xlabel('Frames Seen', fontsize=9)
        ax4.set_ylabel('Entropy\n(Lower = More Confident)', fontsize=9)
        ax4.set_title('Confidence\nEvolution', fontsize=10, fontweight='bold')
        ax4.grid(True, alpha=0.3)
        ax4.set_xlim(0, len(predictions))

        # Overall title
        fig.suptitle(f'Progressive Prediction Evolution: Frame {t}/{T} ' +
                    f'({t/T*100:.1f}% of data)',
                    fontsize=14, fontweight='bold')

        return [ax1, ax2, ax3, ax4]

    # Create animation
    anim = FuncAnimation(fig, update, frames=len(predictions),
                        interval=1000//fps, blit=False, repeat=True)

    # Save as GIF (Colab-friendly)
    writer = PillowWriter(fps=fps)
    anim.save(output_path, writer=writer)
    print(f"Video saved to {output_path} (GIF format)")

    plt.close()

    return output_path, predictions, confidences

# Generate prediction videos for ALL datasets
if 'best_model' not in locals() or 'datasets' not in locals() or 'all_centers' not in locals():
    print("Error: Required variables not found!")
else:
    print("GENERATING PREDICTION VIDEOS FOR ALL DATASETS")

    # Use OUTPUT_PATH if available, otherwise current directory
    if 'OUTPUT_PATH' in locals():
        output_dir = OUTPUT_PATH
    else:
        output_dir = '.'

    # Generate video for each dataset
    video_results = {}
    
    for dataset_idx, (dataset_name, full_data) in enumerate(datasets.items()):
        print(f"\n{'='*80}")
        print(f"Dataset {dataset_idx + 1}/{len(datasets)}: {dataset_name}")
        print(f"{'='*80}")
        
        # Get sequence and centers for this dataset
        seq = torch.from_numpy(full_data).float().unsqueeze(1)  # Add channel dimension
        
        # Get all aggregation centers for this dataset
        centers_array = all_centers.get(dataset_name, np.array([]))
        if len(centers_array) > 0:
            # Convert from [cx, cy] to (x, y) format
            all_true_centers = [(c[0], c[1]) for c in centers_array]
            true_center = all_true_centers[0]  # Use first center as primary
        else:
            all_true_centers = []
            true_center = (seq.shape[3]//2, seq.shape[2]//2)  # Default to center
        
        print(f"\nDataset Info:")
        print(f"Dataset name: {dataset_name}")
        print(f"Input shape: {seq.shape}")
        print(f"Total frames: {seq.shape[0]}")
        print(f"Aggregation centers: {len(all_true_centers)}")
        if len(all_true_centers) > 0:
            print(f"  - Primary center: ({true_center[0]:.0f}, {true_center[1]:.0f})")

        # Create video filename
        video_filename = f'slime_mold_prediction_{dataset_name}.gif'
        video_path_full = os.path.join(output_dir, video_filename)

        print(f"\nCreating video...")
        
        try:
            video_path, preds, confs = create_prediction_video(
                best_model, seq, true_center, all_true_centers=all_true_centers,
                output_path=video_path_full,
                fps=10
            )

            print(f"\nVideo Statistics:")
            print(f"Total prediction frames: {len(preds)}")
            print(f"Playback speed: 10 fps")
            print(f"Video duration: ~{len(preds)/10:.1f} seconds")
            print(f"Initial confidence (entropy): {confs[0]:.4f}")
            print(f"Final confidence (entropy): {confs[-1]:.4f}")
            print(f"Confidence improvement: {(confs[0]-confs[-1])/confs[0]*100:.1f}%")
            
            video_results[dataset_name] = {
                'path': video_path,
                'frames': len(preds),
                'initial_conf': confs[0],
                'final_conf': confs[-1]
            }
            
            # Display video in notebook
            print(f"\n🎥 Displaying video for {dataset_name}...")
            try:
                from IPython.display import Image as IPImage
                display(IPImage(filename=video_path))
            except:
                display(HTML(f'<img src="{video_path}">'))
            
            print(f"\nVideo saved to: {video_path}")
            
        except Exception as e:
            print(f"\nError generating video for {dataset_name}: {e}")
            import traceback
            traceback.print_exc()

    # Final summary
    print("ALL PREDICTION VIDEOS GENERATED!")
    
    print(f"\nGenerated {len(video_results)} videos:")
    for dataset_name, info in video_results.items():
        print(f"\n  {dataset_name}:")
        print(f"File: {info['path']}")
        print(f"Frames: {info['frames']}")
        print(f"Confidence improvement: {(info['initial_conf']-info['final_conf'])/info['initial_conf']*100:.1f}%")
    
    print("\nEach video shows:")
    print("1. Input Frame: Current observation at time t")
    print("2. Prediction Heatmap: Model's aggregation belief (with ALL centers)")
    print("3. Motion Flow: Optical flow vectors (movement cues)")
    print("4. Confidence: Entropy evolution (lower = more certain)")